# Importing useful libraries

In [1]:
!apt-get install poppler-utils
!apt-get install tesseract-ocr
!apt-get install libtesseract-dev
!pip install pytesseract
!pip install pdf2image

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.3 [186 kB]
Fetched 186 kB in 0s (508 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121658 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.3_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.3) ...
Setting up poppler-utils (22.02.0-2ubuntu0.3) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
T

In [2]:
import random
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from PIL import Image
from io import BytesIO
import pytesseract
import pdf2image
from pdf2image import convert_from_path
import os

# Download NLTK data
nltk.download('punkt')

# Set the path to Poppler binaries
poppler_path = "/usr/bin"  # Use the default path in Colab

# Optionally, set the environment variable
os.environ["PYPDF2_BIN_PATH"] = poppler_path
os.environ["PATH"] += os.pathsep + poppler_path

pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

# Rest of your code...




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Function to read PDF files that returns text corpus.

In [3]:
# Function to read PDF files that returns text corpus.
def get_pdf_data(user_resp):
    PDF_file = "pdf1.pdf"
    pages = convert_from_path(PDF_file, 500, poppler_path=poppler_path)
    image_counter = 1
    for page in pages:
        filename = "page_" + str(image_counter) + ".jpg"
        page.save(filename, 'JPEG')
        image_counter = image_counter + 1

    filelimit = image_counter - 1
    corpus = ''
    for i in range(1, filelimit + 1):
        filename = "page_" + str(i) + ".jpg"
        text = str(((pytesseract.image_to_string(Image.open(filename)))))
        text = text.replace('-\n', '')
        corpus += text

    sent_tokens = nltk.sent_tokenize(corpus)
    return sent_tokens


#  Cleaning the data and making it usable.

In [4]:
# Cleaning the data and making it usable.
def LemNormalize(corpus):
    return nltk.word_tokenize(corpus.lower().translate(str.maketrans('', '', string.punctuation)))

# Greeting Inputs
GREETING_INPUTS = ["hi", "hello", "hola", "greetings", "wassup", "hey"]
# Greeting responses back to the user
GREETING_RESPONSES=["howdy", "hi", "hey", "what's good", "hello", "hey there"]
# Function to return a random greeting response to a user's greeting
def greeting(sentence):
    # If the user's input is a greeting, then return a randomly chosen greeting response
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

# Create a text similarity detection function that matches the user inputs and returns similar sentences.

## Here I have used TF-IDF vectors and cosine similarity scores for matching the data with user-input


In [5]:
# Create a text similarity detection function that matches the user inputs and returns similar sentences.
def response(user_response):
    # The user's response / query
    user_response = user_response.lower()  # Make the response lowercase
    # Set the chatbot response to an empty string
    robo_response = ''
    sent_tokens = get_pdf_data(user_response)
    # Append the user's response to the sentence list
    sent_tokens.append(user_response)
    # Create a TfidfVectorizer Object
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    # Convert the text to a matrix of TF-IDF features
    tfidf = TfidfVec.fit_transform(sent_tokens)
    # Get the measure of similarity (similarity scores)
    vals = cosine_similarity(tfidf[-1], tfidf)
    # Get the index of the most similar text/sentence to the user's response
    try:
        idx = vals.argsort()[0][-2]
    except IndexError:
        robo_response = robo_response + "I apologize, I don't understand."
        return robo_response
    # Reduce the dimensionality of vals
    flat = vals.flatten()
    # Sort the list in ascending order
    flat.sort()
    # Get the most similar score to the user's response
    score = flat[-2]
    # If the variable 'score' is 0 then there is no text similar to the user's response
    if score == 0:
        robo_response = robo_response + "I apologize, I don't understand."
    else:
        robo_response = robo_response + sent_tokens[idx]

    # Remove the user's response from the sentence tokens list
    sent_tokens.remove(user_response)

    return robo_response

## A function that handles the input of the users.

In [9]:

# A function that handles the input of the users.
flag = True
print("ChatBot: Hi! I will answer your queries. Please Ask. If you want to exit, type Bye!")
while flag:
    user_response = input()
    user_response = user_response.lower()
    if user_response != 'bye':
        if user_response == 'thanks' or user_response == 'thank you':
            flag = False
            print("ChatBot: You are welcome!")
        else:
            if greeting(user_response) is not None:
                print("ChatBot: " + greeting(user_response))
            else:
                print("ChatBot: " + response(user_response))
    else:
        flag = False
        print("ChatBot: Chat with you later!")

ChatBot: Hi! I will answer your queries. Please Ask. If you want to exit, type Bye!
count oranges
ChatBot: Oranges: There are a total of 7,200 oranges over 71 images
giving on average 102 oranges per image.
what is accuracy of labels?
ChatBot: While a simple accuracy, or
mean error, is appealing because it is easy to interpret, it is
misleading because a model can have high accuracy over the
entire data set, but low accuracy per image since the errors
will wash each other out.
bye
ChatBot: Chat with you later!
